In [41]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import os
import pandas as pd
import datetime
import numpy as np
import xlsxwriter
import calendar
from calendar import monthrange
from selectfile import FileBrowser
from utils import staff_code

## Crear time sheet segun Arquitecto y segun mes y año

In [42]:
## define list of weeks, of a given month
# return 2 lists, 1 with the dates object on each week and the other with the name of the folder of that week
def weeks_list(year = datetime.date.today().year , month =  datetime.date.today().month):
    todayDate  = datetime.date.today().replace(day = 1 , month = month, year = year)
    cal= calendar.Calendar()
    weeks_date_full = cal.monthdatescalendar(year, month)
    weeks_date = []
    folders_names = []
    
    for i , week in enumerate(weeks_date_full) :
        weeks_date.append([day for day in week if day.month == todayDate.month])

    for week in weeks_date :     
        folders_names.append(week[0].strftime('%Y-%m%d-') + week[-1].strftime('%m%d'))
    return weeks_date , folders_names

In [43]:
def get_proyects (path ='Project Board.xls' , number = 7 ):
    projects_df = pd.read_excel('Project Board.xls' , sheet_name = 'Project Board')
    projects_df.columns = projects_df.iloc[5]
    projects_df = projects_df.iloc[6: , :5]
    projects_df = projects_df.dropna(subset=['PROJECT NAME'])
    return projects_df[-number:]

In [44]:
def get_last_proyects():
    last_projects = []
    for row in get_proyects().loc[:, ['CODE','PROJECT NAME + CODIGO EXTERNO']].iterrows():
        last_projects.append([row[1][0] ,row[1][1]])
    return last_projects

In [45]:
# Create time sheet for a given dates and person
# date_string : string name of the week
# date_list : list of dates to add on the excel
# names: tuple of number_code and name of person
def time_sheet(path, date_string, date_list , names):
    title = names[0] + '-'+ names[1] + '-'+ date_string +'.xlsx'
    file_name = os.path.join(path, title)
    if not os.path.exists(file_name):
        workbook = xlsxwriter.Workbook(file_name)
        worksheet = workbook.add_worksheet('Time Sheet')
        fxc_ws = workbook.add_worksheet('FXC Timesheet Format')

        # Increase the cell size of the merged cells to highlight the formatting.
        worksheet.set_column(0,0, 15)
        worksheet.set_column(6,6, 12)
        worksheet.set_column('B:Z', 7)
        worksheet.set_row(0, 1)
        worksheet.set_row(2, 1)
        worksheet.set_row(1, 20)
        worksheet.freeze_panes(14, 0)

        # Create a format to use in the merged range.
        merge_format_gray = workbook.add_format({'bold': 1,'border': 1,'align': 'center','valign': 'vcenter',
                                                 'font_size' : 14 ,'fg_color': '#C0C0C0'})

        merge_format_blue = workbook.add_format({'bold': 1,'border': 1,'align': 'center','valign': 'vcenter',
                                                 'font_size' : 16 ,'font_color' : 'white','fg_color': '#191970'})

        format_bold_center = workbook.add_format({'bold': 1,'border': 1,'align': 'center','valign': 'vcenter',
                                                  'font_size' : 11 })

        format_center = workbook.add_format({'border': 1,'align': 'center','valign': 'vcenter','font_size' : 10 })

        format_header = workbook.add_format({'bold': 1, 'border': 1,'align': 'center','valign': 'vcenter',
                                             'fg_color': '#191970','font_color' : 'white' ,'font_size' : 12 })

        format_time1 = workbook.add_format({'num_format': 'hh:mm', 'align': 'center','valign': 'vcenter', 
                                            'fg_color': '#DCE6F1','font_size' : 10, 'left': 1,'right' : 1,})

        format_time2 = workbook.add_format({'num_format': 'hh:mm', 'align': 'center','valign': 'vcenter', 
                                            'fg_color': '#B8CCE4','font_size' : 10, 'left': 1,'right' : 1,})

        format_sheet1 = workbook.add_format({'left': 1,'right' : 1,'align': 'center','valign': 'vcenter',
                                             'fg_color': '#DCE6F1','font_size' : 10 })

        format_sheet2 = workbook.add_format({'left': 1,'right' : 1,'align': 'center','valign': 'vcenter',
                                             'fg_color': '#B8CCE4','font_size' : 10 })

        format1 = workbook.add_format({'bg_color': 'white', 'border': 6,'align': 'center', 'valign': 'vcenter'})

        format2 = workbook.add_format({'bg_color': '#C0C0C0', 'border': 6,'align': 'center', 'valign': 'vcenter'})
        
        format_bold_center_fxc = workbook.add_format({'bg_color': '#C0C0C0','bold': 1,'border': 1,'align': 'center'
                                                      ,'valign': 'vcenter', 'font_size' : 11 })
        format_bold_center_fxc.set_text_wrap()

        worksheet.merge_range('B2:C2', 'NOMBRE', merge_format_gray)
        worksheet.merge_range('D2:E2', names[0], merge_format_blue)
        worksheet.merge_range('F2:K2', names[1] ,merge_format_blue)
        worksheet.merge_range('B4:H4', 'WORK SUMMARY' ,format_bold_center)
        worksheet.merge_range('I4:L5', 'LABOR CODE', format_bold_center )
        worksheet.merge_range('I13:L13', 'TOTAL HOURS' ,format_bold_center )
        worksheet.merge_range('M13:O13', '=SUM(M6:M12)' ,format_bold_center )
        worksheet.merge_range('C5:H5', 'Project Name', format_center )
        worksheet.write('B5', 'Proj. #' , format_center)
        worksheet.merge_range('M4:O5', 'TOTAL HOURS'  ,format_bold_center)
        worksheet.merge_range('R4:X5', 'ULTIMOS PROYECTOS INCORPORADOS', format_bold_center)
        worksheet.add_table('R6:X12', {'data': [], 'header_row': False})

        get_last_proy = '=LOOKUP(2,1/(\'[Project Board.xls]Project Board\'!$C$8:$C$600<>""),\'[Project Board.xls]Project Board\'!$A$8:$A$600)'
        index_last_proy = '=MATCH(R6,\'[Project Board.xls]Project Board\'!$A$8:$A$600)'
        worksheet.write('R6', get_last_proy )
        worksheet.write('X6', index_last_proy )
        worksheet.write('S6','=VLOOKUP(R6,\'[Project Board.xls]Project Board\'!$A$8:$E$600,5,)')
        for i in range(6):
            n_last_proy = '=INDEX(\'[Project Board.xls]Project Board\'!$A$8:$A$600,X6-'+ str(i + 1) + ',0)'
            worksheet.write('R'+str(i+7), n_last_proy )
            worksheet.write('S'+str(i+7),'=VLOOKUP(R'+str(i+7)+',\'[Project Board.xls]Project Board\'!$A$8:$E$600,5,)')

        worksheet.write_url('R14', "external:c:/Users/Fabian/Documents/Development/libovich/create_time_shee/Project Board.xls",  string='>> Ir a Project Board')

        for i in range(6,13):
            if i%2 :
                worksheet.write('B'+str(i), '' , format1)
                worksheet.write_formula('C'+str(i), '=IF(B'+str(i)+'<>"",VLOOKUP(B'+str(i)+',\'[Project Board.xls]Project Board\'!$A$8:$E$600,2,),"")' , format1)
                worksheet.merge_range(str('D'+str(i)+':H'+str(i)),'=IF(B'+str(i)+'<>"",VLOOKUP(B'+str(i)+',\'[Project Board.xls]Project Board\'!$A$8:$E$600,5,),"")', format1)
                worksheet.merge_range(str('I'+str(i)+':L'+str(i)),'', format1)
                worksheet.merge_range(str('M'+str(i)+':O'+str(i)),'=SUMIF(G15:G60, D'+str(i)+', F15:F60)', format1)

            else :
                worksheet.write('B'+str(i), '' , format2)
                worksheet.write_formula('C'+str(i), '=IF(B'+str(i)+'<>"",VLOOKUP(B'+str(i)+',\'[Project Board.xls]Project Board\'!$A$8:$E$600,2,),"")' , format2)
                worksheet.merge_range(str('D'+str(i)+':H'+str(i)),'=IF(B'+str(i)+'<>"",VLOOKUP(B'+str(i)+',\'[Project Board.xls]Project Board\'!$A$8:$E$600,5,),"")', format2)
                worksheet.merge_range(str('I'+str(i)+':L'+str(i)),'', format2)
                worksheet.merge_range(str('M'+str(i)+':O'+str(i)),'=SUMIF(G15:G60, D'+str(i)+', F15:F60)', format2)

        worksheet.write('A14', 'Weekday' , format_header)
        worksheet.merge_range('B14:C14', 'Date' , format_header)
        worksheet.write('D14', 'In' , format_header)
        worksheet.write('E14', 'Out' , format_header)
        worksheet.write('F14', 'Hours' , format_header)
        worksheet.write('G14', 'Project' , format_header)
        worksheet.merge_range('H14:I14', 'Description' , format_header)

        for i ,date in enumerate(date_list):
            worksheet.write('A'+str(15+i*6), date.strftime("%A"), format_bold_center)
            if i%2:
                for j in range(6):
                    worksheet.write('A'+str(15+i*6+j+1), '', format_sheet1)
                    worksheet.merge_range('B'+str(15+i*6+j)+':C'+str(15+i*6+j), date.strftime("%m/%d/%Y"),format_sheet1)
                    worksheet.write('D'+str(15+i*6+j),'',format_time1)
                    worksheet.write('E'+str(15+i*6+j),'',format_time1)
                    hours_str = '=IF(D'+str(15+i*6+j) + '="","",IF((E'+str(15+i*6+j) +'-D'+str(15+i*6+j) +')*24<0,"0",(E'+str(15+i*6+j) +'-D'+str(15+i*6+j) +')*24))'
                    worksheet.write_formula('F'+str(15+i*6+j),hours_str,format_sheet1)
                    worksheet.write('G'+str(15+i*6+j), '',format_sheet1)
                    worksheet.data_validation('G'+str(15+i*6+j), {'validate': 'list', 'source': '=D6:D12', 'input_message': 'Seleccione un proyecto'})
                    worksheet.merge_range('H'+str(15+i*6+j)+':I'+str(15+i*6+j), '',format_sheet1)
            else:
                for j in range(6):
                    worksheet.write('A'+str(15+i*6+j+1), '', format_sheet2)
                    worksheet.merge_range('B'+str(15+i*6+j)+':C'+str(15+i*6+j), date.strftime("%m/%d/%Y") ,format_sheet2)
                    worksheet.write('D'+str(15+i*6+j),'',format_time2)
                    worksheet.write('E'+str(15+i*6+j),'',format_time2)
                    hours_str = '=IF(D'+str(15+i*6+j) + '="","",IF((E'+str(15+i*6+j) +'-D'+str(15+i*6+j) +')*24<0,"0",(E'+str(15+i*6+j) +'-D'+str(15+i*6+j) +')*24))'
                    worksheet.write_formula('F'+str(15+i*6+j),hours_str,format_sheet2)
                    worksheet.write('G'+str(15+i*6+j), '',format_sheet2)
                    worksheet.data_validation('G'+str(15+i*6+j), {'validate': 'list', 'source': '=D6:D12', 'input_message': 'Seleccione un proyecto'})
                    worksheet.merge_range('H'+str(15+i*6+j)+':I'+str(15+i*6+j), '',format_sheet2)

        worksheet.write('A'+str(15+i*6+j+1), '' , format_center)
        fxc_ws.set_column(0,0,20)
        fxc_ws.set_column(3,3,25)
        fxc_ws.set_row(3,30)
        
        fxc_ws.write('A1','Timesheet FXC')
        fxc_ws.write('A2','FXC')
        fxc_ws.write('A4','Employee',format_bold_center_fxc)
        fxc_ws.write('B4','Company',format_bold_center_fxc)
        fxc_ws.write('C4','FXC Project Number',format_bold_center_fxc)
        fxc_ws.write('D4','FXC Project Name',format_bold_center_fxc)
        fxc_ws.write('E4','Task',format_bold_center_fxc)
        fxc_ws.write('F4','Task Name',format_bold_center_fxc)
        fxc_ws.write('G4','SubTask',format_bold_center_fxc)
        fxc_ws.write('H4','SubTask Name',format_bold_center_fxc)
        fxc_ws.write('I4','Labor Code',format_bold_center_fxc)
        fxc_ws.write('J4','Total Hours',format_bold_center_fxc)
        letters_dates = ['K','L','M','N','O','P','Q','R','S','T']
        for i ,date in enumerate(date_list):
            fxc_ws.write(letters_dates[i] + '4',date.strftime("%a %m/%d"),format_bold_center_fxc)
        
        for i in range(5,12):
            fxc_ws.write('A'+str(i) ,'=\'Time Sheet\'!$F$2')
            fxc_ws.write('B'+str(i) ,'=IF(\'Time Sheet\'!C'+str(i+1)+' = "FXC", "FXC","")')
            fxc_ws.write('D'+str(i) ,'=IF(B'+str(i)+' <>"",\'Time Sheet\'!D'+str(i+1)+':H'+str(i+1)+',"")')
            fxc_ws.write('J'+str(i) ,'=IF(B'+str(i)+'="FXC",SUM(K'+str(i)+':Q'+str(i)+'),0)')
            for j in range(len(date_list)):
                cell_form = '=IF(B'+str(i)+'<>"",SUMIF(\'Time Sheet\'!$G$' +str(15+j*6)+':$G$'+str(20+j*6)+',\'FXC Timesheet Format\'!D'+str(i)+',\'Time Sheet\'!$F$' +str(15+j*6)+':$F$'+str(20+j*6)+'),0)'
                fxc_ws.write(letters_dates[j] + str(i) , cell_form)
        workbook.close()
        with output:
                print('Time sheet creado:' + title)
    else :
        with output:
                print('Time sheet:' + title + ' existe')

In [46]:
def creat_excels(b):
    if staff_picker.value == 'Todos':
        staff_path = os.path.join('O:','Libovich','F','Estudio','Time Sheets','Staff.xls')
        staff_list = staff_code(path = staff_path)
    else:
        staff_list = [tuple(staff_picker.value.split(' - '))]
    
    date_picked = list(map(int,date_picker.value.split('-')))
    
    folder_path = os.path.join('O:','Libovich','F','Estudio','Time Sheets')

    for i, week in enumerate(weeks_list(date_picked[0], date_picked[1])[1]):
        if os.path.isdir(os.path.join(folder_path , week)):
            with output:
                print('carpeta '+ week + ' existe')
        else :
            os.makedirs(os.path.join(folder_path , week))
        path = os.path.join(folder_path, week)
        for staff_name in staff_list:
            time_sheet(path ,week ,weeks_list(date_picked[0], date_picked[1])[0][i], staff_name )

In [47]:
# interfaz
dates_range = pd.date_range(datetime.date.today() + datetime.timedelta(days=-61),datetime.date.today() + datetime.timedelta(days=+61), freq='MS')

date_picker = widgets.Dropdown(
    options=dates_range.strftime("%Y-%m").tolist(),
    # value='2',
    description='Año - mes',
    disabled=False,
) 
#staff_path = os.path.join('O:','Libovich','F','Estudio','Time Sheets','Staff.xls')
staffs = [' - '.join(i) for i in staff_code()] 
staffs.insert(0,'Todos')
staff_picker = widgets.Dropdown(
    options=staffs,
    # value='2',
    description='Staff',
    disabled=False,
)

button = widgets.Button(description="Crear time sheets")
output = widgets.Output()
button.on_click(creat_excels)

widgets.HBox([
    staff_picker,
    date_picker,
    button
    ],
    layout = widgets.Layout(width = '900px')
)

In [48]:
display(output)

Output()